**ARTIFICIAL INTELLIGENCE & CHATBOT**

**CHECKPOINT 6** - Análise Exploratória e Modelo Não Supervisionado

**PROF:** Felipe Souza Amaral

**ALUNA:** Marina Yumi Kanadani | **RM:** 558404 | **TURMA:** 1TDSPX

**FIAP | NOVEMBRO | 2024**

# **IGN GAMES FROM BEST TO WORST**

**ESCOLHA DO DATASET**

Escolhi o dataset do Kaggle "IGN games from best to worst" por interesse pessoal e pelas características:
- *Usability:* 10.00
- *Update:* 07/11/2024
- *Densidade de Dados:* 12589 valores únicos de títulos de jogos

**DEFINIÇÃO DOS OBJETIVOS**
- Agrupar jogos segundo gênero, plataforma e nota: identificar segmentos de jogos com características similares.
- Agrupar jogos segundo gênero e data de lançamento: observar quais gêneros se popularizaram mais em determinados períodos.
- Agrupar jogos segundo gênero e nota: avaliar relação entre gêneros e notas.

# **ANÁLISE EXPLORATÓRIA**

**BIBLIOTECAS**

Iniciaremos carregando as bibliotecas necessárias.

In [87]:
# Análise e manipulação de dados
import pandas as pd
import numpy as np

# Construção de gráficos
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import colors
import seaborn as sns
import scipy.cluster.hierarchy as sch

# Algoritmos de Agrupamento
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

# Avaliação de desempenho
from sklearn.metrics import adjusted_rand_score, silhouette_score

**DATASET**

Agora, vamos carregar o dataset e exibir as primeiras linhas.

In [88]:
# Carrega o dataset
games = pd.read_csv('ign-games.csv')
games.head()

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
0,Checkered Flag,10.0,Masterpiece,Lynx,Racing,1999,7,6
1,Chrono Trigger,10.0,Masterpiece,Wii,"Action, RPG",2011,5,25
2,Dragon Warrior III,10.0,Masterpiece,Game Boy Color,RPG,2001,7,20
3,Grand Theft Auto IV,10.0,Masterpiece,Xbox 360,"Action, Adventure",2008,4,25
4,Grand Theft Auto IV,10.0,Masterpiece,PlayStation 3,"Action, Adventure",2008,4,25


**TRADUÇÃO**

Vamos traduzir os nomes das colunas e das variáveis categórias (exceto títulos e plataformas, que não precisam ser traduzidos).

Primeiro, vamos separar as colunas das variáveis categóricas cujos valores desejamos traduzir: 'score_phrase' e 'genre'.

In [89]:
# Obtém os valores únicos para 'score_phrase' apenas
categorias_unicas_notas = games['score_phrase'].unique()

# Exibe as categorias únicas de notas
print("Categorias únicas de notas encontradas:")
print(categorias_unicas_notas)

# Cria um conjunto para armazenar todos os gêneros únicos
generos_unicos = set()

# Itera sobre os valores na coluna 'genre' e adiciona gêneros únicos ao conjunto
for generos in games['genre'].dropna():
    for genero in generos.split(', '):
        generos_unicos.add(genero.strip())

# Exibe os gêneros únicos
print("Gêneros únicos encontrados:")
print(np.array(list(generos_unicos)))

Categorias únicas de notas encontradas:
['Masterpiece' 'Amazing' 'Great' 'Good' 'Okay' 'Mediocre' 'Bad' 'Awful'
 'Painful' 'Unbearable' 'Disaster']
Gêneros únicos encontrados:
['Party' 'Racing' 'Puzzle' 'Hardware' 'Shooter' 'Trivia' 'Other'
 'Strategy' 'Sports' 'Virtual Pet' 'Music' 'Card' 'Word Game' 'Adult'
 'Wrestling' 'Battle' 'Adventure' 'RPG' 'Compilation' 'Board' 'Golf'
 'Hunting' 'Episodic' 'Platformer' 'Productivity' 'Flight' 'Pinball'
 'Simulation' 'Action' 'Casino' 'Editor' 'Educational' 'Fighting'
 'Baseball' 'First-Person']


Agora, faremos os dicionários necessários.

In [90]:
# Dicionário para traduzir as colunas
traducao_colunas = {
    'title': 'Título',
    'score': 'Nota',
    'score_phrase': 'Categoria da Nota',
    'platform': 'Plataforma',
    'genre': 'Gênero',
    'release_year': 'Ano de Lançamento',
    'release_month': 'Mês de Lançamento',
    'release_day': 'Dia de Lançamento',
}

# Dicionários para traduzir os valores das variáveis categóricas
traducao_categoria_nota = {
    'Masterpiece': 'Obra-prima',
    'Amazing': 'Incrível',
    'Great': 'Ótimo',
    'Good': 'Bom',
    'Okay': 'Ok',
    'Mediocre': 'Medíocre',
    'Bad': 'Ruim',
    'Awful': 'Horrível',
    'Painful': 'Doloroso',
    'Unbearable': 'Insuportável',
    'Disaster': 'Desastre',
}

traducao_genero = {
    'Party': 'Multijogador',
    'Racing': 'Corrida',
    'Puzzle': 'Quebra-cabeça',
    'Shooter': 'Tiro',
    'Trivia': 'Curiosidades',
    'Strategy': 'Estratégia',
    'Sports': 'Esportes',
    'Virtual Pet': 'Pet Virtual',
    'Music': 'Música',
    'Card': 'Cartas',
    'Word Game': 'Jogo de Palavras',
    'Adult': 'Adulto',
    'Wrestling': 'Luta Livre',
    'Battle': 'Batalha',
    'Adventure': 'Aventura',
    'Compilation': 'Compilação',
    'Board': 'Tabuleiro',
    'Golf': 'Golfe',
    'Hunting': 'Caça',
    'Episodic': 'Episódico',
    'Platformer': 'Plataforma',
    'Productivity': 'Produtividade',
    'Flight': 'Voo',
    'Pinball': 'Pinball',
    'Simulation': 'Simulação',
    'Action': 'Ação',
    'Casino': 'Cassino',
    'Editor': 'Editor',
    'Educational': 'Educativo',
    'Fighting': 'Luta',
    'Baseball': 'Beisebol',
    'First-Person': 'Primeira Pessoa',
}

Finalmente, vamos substituir os valores com as traduções e visualizar as primeiras linhas alteradas.

In [91]:
# Função para traduzir cada gênero, incluindo gêneros compostos
def traduzir_genero(genero):
    if pd.isna(genero):
        return genero
    return ', '.join([traducao_genero.get(g.strip(), g.strip()) for g in genero.split(',')])

# Aplica as traduções na coluna 'score_phrase'
games['score_phrase'] = games['score_phrase'].replace(traducao_categoria_nota)

# Aplica as traduções na coluna 'genre' usando a função personalizada para gêneros compostos
games['genre'] = games['genre'].apply(traduzir_genero)

# Aplica as traduções nos nomes das colunas
games.rename(columns=traducao_colunas, inplace=True)

# Exibe as primeiras linhas do dataset
print(games.head())

                Título  Nota Categoria da Nota      Plataforma  \
0       Checkered Flag  10.0        Obra-prima            Lynx   
1       Chrono Trigger  10.0        Obra-prima             Wii   
2   Dragon Warrior III  10.0        Obra-prima  Game Boy Color   
3  Grand Theft Auto IV  10.0        Obra-prima        Xbox 360   
4  Grand Theft Auto IV  10.0        Obra-prima   PlayStation 3   

           Gênero  Ano de Lançamento  Mês de Lançamento  Dia de Lançamento  
0         Corrida               1999                  7                  6  
1       Ação, RPG               2011                  5                 25  
2             RPG               2001                  7                 20  
3  Ação, Aventura               2008                  4                 25  
4  Ação, Aventura               2008                  4                 25  


**DESCRIÇÃO DOS DADOS**

Vamos exibir o número de linhas e colunas, seguido dos tipos de variáveis presentes.

In [97]:
# Número de linhas e colunas
num_linhas, num_colunas = games.shape
print(f"Número de linhas: {num_linhas}, Número de colunas: {num_colunas}")

# Descrição dos dados
print("\nInformações do dataset:")
games.info()

# Exibe colunas numéricas
colunas_numericas = games.select_dtypes(include=['number']).columns
print("\nColunas numéricas:", colunas_numericas)

# Exibe colunas categóricas
colunas_categoricas = games.select_dtypes(include=['object']).columns
print("\nColunas categóricas:", colunas_categoricas)

Número de linhas: 18589, Número de colunas: 8

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 18589 entries, 0 to 18624
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Título             18589 non-null  object 
 1   Nota               18589 non-null  float64
 2   Categoria da Nota  18589 non-null  object 
 3   Plataforma         18589 non-null  object 
 4   Gênero             18589 non-null  object 
 5   Ano de Lançamento  18589 non-null  int64  
 6   Mês de Lançamento  18589 non-null  int64  
 7   Dia de Lançamento  18589 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 1.3+ MB

Colunas numéricas: Index(['Nota', 'Ano de Lançamento', 'Mês de Lançamento', 'Dia de Lançamento'], dtype='object')

Colunas categóricas: Index(['Título', 'Categoria da Nota', 'Plataforma', 'Gênero'], dtype='object')


**DADOS FALTANTES**

A seguir, vamos identificar e tratar os dados faltantes.

In [94]:
# Quantidade de células vazias por coluna
print("Presença de células vazias:")
print(games.isnull().sum())

Presença de células vazias:
Título                0
Nota                  0
Categoria da Nota     0
Plataforma            0
Gênero               36
Ano de Lançamento     0
Mês de Lançamento     0
Dia de Lançamento     0
dtype: int64


Considerando que há apenas 36 dados faltantes na coluna "Gênero", o que corresponde a 0,2% do dataset, optei por remover estas linhas.

In [96]:
# Remoção dos dados faltantes
games.dropna(subset=['Gênero'], inplace=True)

**ANÁLISE DESCRITIVA**

Por fim, vamos realizar a análise descritiva dos dados. Inicialmente, avaliaremos as variáveis numéricas.

In [109]:
# Estatísticas para variáveis numéricas
print("Estatísticas para variáveis numéricas:")
print(games.describe())

# Média (arredondada para datas e convertida para inteiros)
print("\nMédia:")
media = games.mean(numeric_only=True)
print(media[['Nota']].append(media[['Ano de Lançamento', 'Mês de Lançamento', 'Dia de Lançamento']].round(0).astype(int)))

# Mediana (arredondada para datas e convertida para inteiros)
print("\nMediana:")
mediana = games.median(numeric_only=True)
print(mediana[['Nota']].append(mediana[['Ano de Lançamento', 'Mês de Lançamento', 'Dia de Lançamento']].round(0).astype(int)))

# Máximo (exibindo todas as colunas como inteiros onde aplicável)
print("\nMáximo:")
maximo = games[['Nota']].max().append(games[['Ano de Lançamento', 'Mês de Lançamento', 'Dia de Lançamento']].max().astype(int))
print(maximo)

# Desvio Padrão (arredondado apenas para exibição)
print("\nDesvio Padrão:")
desvio_padrao = games.std(numeric_only=True)
print(desvio_padrao[['Nota']].append(desvio_padrao[['Ano de Lançamento', 'Mês de Lançamento', 'Dia de Lançamento']].round(2)))



Estatísticas para variáveis numéricas:
               Nota  Ano de Lançamento  Mês de Lançamento  Dia de Lançamento
count  18589.000000       18589.000000       18589.000000       18589.000000
mean       6.950853        2006.508634           7.139868          15.601431
std        1.712078           4.588687           3.477048           8.692460
min        0.500000        1970.000000           1.000000           1.000000
25%        6.000000        2003.000000           4.000000           8.000000
50%        7.300000        2007.000000           8.000000          16.000000
75%        8.200000        2010.000000          10.000000          23.000000
max       10.000000        2016.000000          12.000000          31.000000

Média:
Ano de Lançamento    2007
Mês de Lançamento       7
Dia de Lançamento      16
dtype: int64

Mediana:
Ano de Lançamento    2007
Mês de Lançamento       8
Dia de Lançamento      16
dtype: int64

Máximo:
Nota                   10
Ano de Lançamento    2016
Mês de 

Agora, veremos as variáveis categóricas.

In [101]:
# Distribuição de frequências para variáveis categóricas
print("\nDistribuição de frequências para variáveis categóricas:")

# Frequência de `score_phrase`
print("\nScore Phrase:")
print(games['Categoria da Nota'].value_counts())

# Frequência de `platform`
print("\nPlatform:")
print(games['Plataforma'].value_counts())

# Frequência de `genre`
print("\nGenre:")
print(games['Gênero'].value_counts())



Distribuição de frequências para variáveis categóricas:

Score Phrase:
Categoria da Nota
Ótimo           4765
Bom             4725
Ok              2940
Medíocre        1957
Incrível        1803
Ruim            1268
Horrível         662
Doloroso         339
Insuportável      72
Obra-prima        55
Desastre           3
Name: count, dtype: int64

Platform:
Plataforma
PC                      3367
PlayStation 2           1684
Xbox 360                1631
Wii                     1362
PlayStation 3           1355
Nintendo DS             1044
PlayStation              952
Wireless                 906
iPhone                   828
Xbox                     821
PlayStation Portable     632
Game Boy Advance         623
GameCube                 509
Game Boy Color           356
Nintendo 64              302
Dreamcast                286
PlayStation 4            277
Nintendo DSi             252
Nintendo 3DS             222
Xbox One                 208
PlayStation Vita         155
Wii U                 